In [ ]:
import tensorflow as tf 
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from os.path import join

In [ ]:
!unzip '/content/drive/MyDrive/Face mask/data3.zip'

In [ ]:
base_dir = '/content/Dataset'

from google.colab.patches import cv2_imshow
x_train = []
y_train = []
x_test = []
y_test = []
label = {
    'without_mask':0,
    'with_mask':1
}
x = []
y = []

# for sett in ['test', 'train']:
#     path = join(base_dir, sett)
for c in ['with_mask', 'without_mask']:
    p = join(base_dir, c)
    for fil in os.listdir(p):
        im = join(p, fil)
        img = cv2.imread(im)
        img = cv2.resize(img, (224,224))
        x.append(img)
        y.append(label[c])
        # if sett == 'train':
        #     x_train.append(img)
        #     y_train.append(label[c])
        # else:
        #     x_test.append(img)
        #     y_test.append(label[c])

x = np.array(x)
y = np.array(y)

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [ ]:
x_train = np.array([x_train]).reshape(-1,224,224,3)
y_train = np.array([y_train]).reshape(-1)
x_test = np.array([x_test]).reshape(-1,224,224,3)
y_test = np.array([y_test]).reshape(-1)

In [ ]:
y_train = tf.one_hot(y_train, 2)
y_test = tf.one_hot(y_test, 2)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
	rescale = 1./255,
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest"
)

In [ ]:
from tensorflow.keras.applications import MobileNetV2
import tensorflow.keras.layers as Layers
from tensorflow.keras.models import Model


basemodel = MobileNetV2(weights = 'imagenet', include_top=False, input_tensor = Layers.Input(shape = (224,224,3)))
basemodel.trainable = False
top = basemodel.output
# top = Layers.AveragePooling2D((7,7))(top)
# top = Layers.Flatten()(top)
top = Layers.GlobalAveragePooling2D(keepdims = False)(top)
top = Layers.Dense(512)(top)
top = Layers.LeakyReLU()(top)
top = Layers.Dropout(0.5)(top)
top = Layers.Dense(128)(top)
top = Layers.LeakyReLU()(top)
top = Layers.Dropout(0.5)(top)
top = Layers.Dense(2, activation = 'softmax')(top)


model = Model(inputs=basemodel.input, outputs=top)
# model.summary()

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate = 0.0001)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [ ]:
history = model.fit(aug.flow(x_train, y_train, batch_size=32), validation_data=(x_test, y_test), epochs = 20)

Epoch 1/20
150/150 [==============================] - 65s 431ms/step - loss: 0.0641 - accuracy: 0.9743 - val_loss: 0.7742 - val_accuracy: 0.6937
Epoch 2/20
150/150 [==============================] - 65s 430ms/step - loss: 0.0616 - accuracy: 0.9764 - val_loss: 0.9874 - val_accuracy: 0.6544
Epoch 3/20
150/150 [==============================] - 65s 431ms/step - loss: 0.0572 - accuracy: 0.9783 - val_loss: 0.7328 - val_accuracy: 0.7003
Epoch 4/20
150/150 [==============================] - 63s 422ms/step - loss: 0.0574 - accuracy: 0.9772 - val_loss: 0.7879 - val_accuracy: 0.6778
Epoch 5/20
150/150 [==============================] - 59s 393ms/step - loss: 0.0621 - accuracy: 0.9747 - val_loss: 0.8800 - val_accuracy: 0.6870
Epoch 6/20
150/150 [==============================] - 59s 395ms/step - loss: 0.0620 - accuracy: 0.9754 - val_loss: 0.9933 - val_accuracy: 0.6461
Epoch 7/20
150/150 [==============================] - 59s 392ms/step - loss: 0.0594 - accuracy: 0.9777 - val_loss: 0.9600 - val_ac

In [ ]:
from sklearn.metrics import classification_report

preds = np.argmax(model.predict(x_test, batch_size=32), axis = -1)
print(classification_report(np.argmax(y_test, axis=1), preds, target_names=['without_mask', 'with_mask']))

              precision    recall  f1-score   support

without_mask       0.77      0.43      0.55       576
   with_mask       0.63      0.88      0.73       622

    accuracy                           0.66      1198
   macro avg       0.70      0.66      0.64      1198
weighted avg       0.69      0.66      0.65      1198



In [ ]:
model.save('mask_detection2.h5')

/usr/local/lib/python3.7/dist-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
